In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
import datetime as dt
import cv2
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2

from google.colab import drive
drive.mount('/content/drive')
# !ls /content/drive/Othercomputers/My\ Laptop/Thesis/UCF101

DATASET_PATH = '/content/drive/Othercomputers/My Laptop/Thesis/UCF50'
# DATASET_PATH = '/content/drive/MyDrive/Thesis/UCF50'
ALL_CLASSES = os.listdir(DATASET_PATH)
# ALL_CLASSES = ALL_CLASSES[:10]
FRAME_HEIGHT = 60
FRAME_WIDTH = 40
GRADIENT_KERNEL_SIZE = 3
NO_OF_FRAMES = 7

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [2]:
def frames_extraction(video_path):
  frames_list = []
  video_reader = cv2.VideoCapture(video_path)
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frames_window = max(int(video_frames_count/NO_OF_FRAMES), 1)
  for frame_counter in range(NO_OF_FRAMES):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
    success, frame = video_reader.read() 
    if not success:
        break
    resized_frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
    normalized_frame = resized_frame / 255
    grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    frames_list.append(grayscale_frame)
  video_reader.release()
  return frames_list

In [3]:
def compute_gradients(frames):
  gradients_x = []
  gradients_y = []

  for i in range(len(frames)):
    gradients_x.append(cv2.Sobel(frames[i], cv2.CV_64F, 1, 0, ksize=GRADIENT_KERNEL_SIZE))
    gradients_y.append(cv2.Sobel(frames[i], cv2.CV_64F, 0, 1, ksize=GRADIENT_KERNEL_SIZE))

  return gradients_x, gradients_y

In [4]:
def compute_optical_flow(frames):
  optical_flow_list_u = []
  optical_flow_list_v = []
  
  for i in range(0, len(frames) - 1):
    flow = cv2.calcOpticalFlowFarneback(frames[i], frames[i+1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow = np.asarray(flow)
    optical_flow_list_u.append(flow[:, :, 0])
    optical_flow_list_v.append(flow[:, :, 1])

  cv2.destroyAllWindows()
  return optical_flow_list_u, optical_flow_list_v 

In [5]:
def create_dataset():
  frames_list = []
  gradient_x_list = []
  gradient_y_list = []
  optical_flow_u_list = []
  optical_flow_v_list = []
  labels_list = []

  for i, category in enumerate(ALL_CLASSES, start=0):
    category_files = os.listdir(DATASET_PATH + '/' + category)
    print(f'Extracting Data of Class: {category}, {i}')
    for name in category_files:
      video_file_path = os.path.join(DATASET_PATH, category, name)
      frames = frames_extraction(video_file_path)
      if len(frames) == NO_OF_FRAMES:
        gradients_x, gradients_y = compute_gradients(frames)
        optical_flow_u, optical_flow_v = compute_optical_flow(frames)
        frames_list.append(frames)
        gradient_x_list.append(gradients_x)
        gradient_y_list.append(gradients_y)
        optical_flow_u_list.append(optical_flow_u)
        optical_flow_v_list.append(optical_flow_v)
        labels_list.append(i)
  return frames_list, gradient_x_list, gradient_y_list, optical_flow_u_list, optical_flow_v_list, labels_list

In [6]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name, x_label, y_label, save_name):
  metric_value_1 = model_training_history.history[metric_name_1]
  metric_value_2 = model_training_history.history[metric_name_2]

  epochs = range(len(metric_value_1))

  plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
  plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)
  plt.title(str(plot_name))

  plt.legend()
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.savefig(save_name)

In [7]:
def create_3d_cnn_model():
  input_size_1 = Input(shape=(7, FRAME_HEIGHT, FRAME_WIDTH, 1))
  input_size_2 = Input(shape=(7, FRAME_HEIGHT, FRAME_WIDTH, 1))
  input_size_3 = Input(shape=(7, FRAME_HEIGHT, FRAME_WIDTH, 1))
  input_size_4 = Input(shape=(6, FRAME_HEIGHT, FRAME_WIDTH, 1))
  input_size_5 = Input(shape=(6, FRAME_HEIGHT, FRAME_WIDTH, 1))


  # layers for frame channel
  c1_1_layer_1 = Conv3D(filters=5, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_1)
  c1_2_layer_1 = Conv3D(filters=5, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_1)
  # layers for gradient_x channel
  c1_1_layer_2 = Conv3D(filters=5, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_2)
  c1_2_layer_2 = Conv3D(filters=5, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_2)
  # layers for gradient_y channel
  c1_1_layer_3 = Conv3D(filters=5, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_3)
  c1_2_layer_3 = Conv3D(filters=5, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_3)
  # layers for optical_flow_u channel
  c1_1_layer_4 = Conv3D(filters=4, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_4)
  c1_2_layer_4 = Conv3D(filters=4, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_4)
  # layers for optical_flow_v channel
  c1_1_layer_5 = Conv3D(filters=4, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_5)
  c1_2_layer_5 = Conv3D(filters=4, kernel_size=(3, 7, 7), activation='relu', data_format="channels_last")(input_size_5)


  # layers for frame channel
  s1_1_layer_1 = Conv3D(filters=5, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_1_layer_1)
  s1_2_layer_1 = Conv3D(filters=5, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_2_layer_1)
  # layers for gradient_x channel
  s1_1_layer_2 = Conv3D(filters=5, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_1_layer_2)
  s1_2_layer_2 = Conv3D(filters=5, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_2_layer_2)
  # layers for gradient_y channel
  s1_1_layer_3 = Conv3D(filters=5, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_1_layer_3)
  s1_2_layer_3 = Conv3D(filters=5, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_2_layer_3)
  # layers for optical_flow_u channel
  s1_1_layer_4 = Conv3D(filters=4, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_1_layer_4)
  s1_2_layer_4 = Conv3D(filters=4, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_2_layer_4)
  # layers for optical_flow_v channel
  s1_1_layer_5 = Conv3D(filters=4, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_1_layer_5)
  s1_2_layer_5 = Conv3D(filters=4, kernel_size=(3, 7, 7), padding='same', activation='relu', data_format="channels_last", strides=(1, 2, 2))(c1_2_layer_5)


  # layers for frame channel
  c2_1_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_1)
  c2_2_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_1)
  c2_3_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_1)
  c2_4_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_1)
  c2_5_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_1)
  c2_6_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_1)
  # layers for gradient_x channel
  c2_1_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_2)
  c2_2_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_2)
  c2_3_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_2)
  c2_4_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_2)
  c2_5_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_2)
  c2_6_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_2)
  # layers for gradient_y channel
  c2_1_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_3)
  c2_2_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_3)
  c2_3_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_3)
  c2_4_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_3)
  c2_5_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_3)
  c2_6_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_3)
  # layers for optical_flow_u channel
  c2_1_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_4)
  c2_2_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_4)
  c2_3_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_4)
  c2_4_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_4)
  c2_5_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_4)
  c2_6_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_4)
  # layers for optical_flow_v channel
  c2_1_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_5)
  c2_2_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_5)
  c2_3_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_1_layer_5)
  c2_4_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_5)
  c2_5_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_5)
  c2_6_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), activation='relu', data_format="channels_last")(s1_2_layer_5)


  # layers for frame channel
  s2_1_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_1_layer_1)
  s2_2_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_2_layer_1)
  s2_3_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_3_layer_1)
  s2_4_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_4_layer_1)
  s2_5_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_5_layer_1)
  s2_6_layer_1 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_6_layer_1)
  # layers for gradient_x channel
  s2_1_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_1_layer_2)
  s2_2_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_2_layer_2)
  s2_3_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_3_layer_2)
  s2_4_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_4_layer_2)
  s2_5_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_5_layer_2)
  s2_6_layer_2 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_6_layer_2)
  # layers for gradient_y channel
  s2_1_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_1_layer_3)
  s2_2_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_2_layer_3)
  s2_3_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_3_layer_3)
  s2_4_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_4_layer_3)
  s2_5_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_5_layer_3)
  s2_6_layer_3 = Conv3D(filters=3, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_6_layer_3)
  # layers for optical_flow_u channel
  s2_1_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_1_layer_4)
  s2_2_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_2_layer_4)
  s2_3_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_3_layer_4)
  s2_4_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_4_layer_4)
  s2_5_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_5_layer_4)
  s2_6_layer_4 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_6_layer_4)
  # layers for optical_flow_v channel
  s2_1_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_1_layer_5)
  s2_2_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_2_layer_5)
  s2_3_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_3_layer_5)
  s2_4_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_4_layer_5)
  s2_5_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_5_layer_5)
  s2_6_layer_5 = Conv3D(filters=2, kernel_size=(3, 7, 6), padding='same', activation='relu', data_format="channels_last", strides=(1, 3, 3))(c2_6_layer_5)


  # layers for frame channel
  merged_layers_1 = concatenate([s2_1_layer_1, s2_2_layer_1, s2_3_layer_1, s2_4_layer_1, s2_5_layer_1, s2_6_layer_1])
  # layers for gradient_x channel
  merged_layers_2 = concatenate([s2_1_layer_2, s2_2_layer_2, s2_3_layer_2, s2_4_layer_2, s2_5_layer_2, s2_6_layer_2])
  # layers for gradient_y channel
  merged_layers_3 = concatenate([s2_1_layer_3, s2_2_layer_3, s2_3_layer_3, s2_4_layer_3, s2_5_layer_3, s2_6_layer_3])
  # layers for optical_flow_u channel
  merged_layers_4 = concatenate([s2_1_layer_4, s2_2_layer_4, s2_3_layer_4, s2_4_layer_4, s2_5_layer_4, s2_6_layer_4])
  # layers for optical_flow_v channel
  merged_layers_5 = concatenate([s2_1_layer_5, s2_2_layer_5, s2_3_layer_5, s2_4_layer_5, s2_5_layer_5, s2_6_layer_5])


  # layers for frame channel
  c3_layer_1 = Conv3D(filters=18, kernel_size=(3, 7, 4), activation='relu', data_format="channels_last")(merged_layers_1)
  # layers for gradient_x channel
  c3_layer_2 = Conv3D(filters=18, kernel_size=(3, 7, 4), activation='relu', data_format="channels_last")(merged_layers_2)
  # layers for gradient_y channel
  c3_layer_3 = Conv3D(filters=18, kernel_size=(3, 7, 4), activation='relu', data_format="channels_last")(merged_layers_3)
  # layers for optical_flow_u channel
  c3_layer_4 = Conv3D(filters=12, kernel_size=(2, 7, 4), activation='relu', data_format="channels_last")(merged_layers_4)
  # layers for optical_flow_v channel
  c3_layer_5 = Conv3D(filters=12, kernel_size=(2, 7, 4), activation='relu', data_format="channels_last")(merged_layers_5)

  final_merged_layer = concatenate([c3_layer_1, c3_layer_2, c3_layer_3, c3_layer_4, c3_layer_5])

  dropout_layer = Dropout(0.4)(final_merged_layer)

  flattened_layer = Flatten()(dropout_layer)
  
  dense_layer = Dense(len(ALL_CLASSES), activation="softmax")(flattened_layer)

  model = Model(inputs=[input_size_1, input_size_2, input_size_3, input_size_4, input_size_5], outputs=dense_layer)
  
  print(model.summary())

  plot_model(model, to_file='3d_cnn_model.png', show_shapes=True, show_layer_names=True)

  return model

In [ ]:
frames, gradient_x, gradient_y, optical_flow_u, optical_flow_v, labels = create_dataset()
encoded_labels = to_categorical(labels)

x_train_frames, x_test_frames, y_train_frames, y_test_frames = train_test_split(frames, encoded_labels)
x_train_frames = tf.convert_to_tensor(x_train_frames, dtype=tf.float32)
x_test_frames = tf.convert_to_tensor(x_test_frames, dtype=tf.float32)
y_train_frames = tf.convert_to_tensor(y_train_frames, dtype=tf.float32)
y_test_frames = tf.convert_to_tensor(y_test_frames, dtype=tf.float32)

x_train_gradient_x, x_test_gradient_x, y_train_gradient_x, y_test_gradient_x = train_test_split(gradient_x, encoded_labels)
x_train_gradient_x = tf.convert_to_tensor(x_train_gradient_x, dtype=tf.float32)
x_test_gradient_x = tf.convert_to_tensor(x_test_gradient_x, dtype=tf.float32)
y_train_gradient_x = tf.convert_to_tensor(y_train_gradient_x, dtype=tf.float32)
y_test_gradient_x = tf.convert_to_tensor(y_test_gradient_x, dtype=tf.float32)

x_train_gradient_y, x_test_gradient_y, y_train_gradient_y, y_test_gradient_y = train_test_split(gradient_y, encoded_labels)
x_train_gradient_y = tf.convert_to_tensor(x_train_gradient_y, dtype=tf.float32)
x_test_gradient_y = tf.convert_to_tensor(x_test_gradient_y, dtype=tf.float32)
y_train_gradient_y = tf.convert_to_tensor(y_train_gradient_y, dtype=tf.float32)
y_test_gradient_y = tf.convert_to_tensor(y_test_gradient_y, dtype=tf.float32)

x_train_optical_flow_u, x_test_optical_flow_u, y_train_optical_flow_u, y_test_optical_flow_u = train_test_split(optical_flow_u, encoded_labels)
x_train_optical_flow_u = tf.convert_to_tensor(x_train_optical_flow_u, dtype=tf.float32)
x_test_optical_flow_u = tf.convert_to_tensor(x_test_optical_flow_u, dtype=tf.float32)
y_train_optical_flow_u = tf.convert_to_tensor(y_train_optical_flow_u, dtype=tf.float32)
y_test_optical_flow_u = tf.convert_to_tensor(y_test_optical_flow_u, dtype=tf.float32)

x_train_optical_flow_v, x_test_optical_flow_v, y_train_optical_flow_v, y_test_optical_flow_v = train_test_split(optical_flow_v, encoded_labels)
x_train_optical_flow_v = tf.convert_to_tensor(x_train_optical_flow_v, dtype=tf.float32)
x_test_optical_flow_v = tf.convert_to_tensor(x_test_optical_flow_v, dtype=tf.float32)
y_train_optical_flow_v = tf.convert_to_tensor(y_train_optical_flow_v, dtype=tf.float32)
y_test_optical_flow_v = tf.convert_to_tensor(y_test_optical_flow_v, dtype=tf.float32)

model = create_3d_cnn_model()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(),
              metrics=tf.keras.metrics.CategoricalAccuracy())
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
model_training_history = model.fit(x=[x_train_frames, x_train_gradient_x, x_train_gradient_y,
                                      x_train_optical_flow_u, x_train_optical_flow_v],
                                   y=[y_train_frames, y_train_gradient_x, y_train_gradient_y,
                                      y_train_optical_flow_u, y_train_optical_flow_v],
                                   epochs=50, batch_size=7, shuffle=True, validation_split=0.2, callbacks=[early_stopping_callback])

plot_metric(model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss', 'Epochs', 'Loss', '3d_cnn_loss_plot')

print('-------------------------Evaluating model now-------------------------')
model_testing_history = model.evaluate(x=[x_test_frames, x_test_gradient_x, x_test_gradient_y, x_test_optical_flow_u,
                                          x_test_optical_flow_v],
                                       y=[y_test_frames, y_test_gradient_x, y_test_gradient_y, y_test_optical_flow_u,
                                          y_test_optical_flow_v])

model_evaluation_loss, model_evaluation_accuracy = model_testing_history

# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'conv3D_model_DateTime_{current_date_time_string}_Loss_{model_evaluation_loss}_Accuracy_{model_evaluation_accuracy}.h5'

# Save your Model.
model.save(model_file_name)

In [ ]:
plot_metric(model_training_history, 'categorical_accuracy', 'val_categorical_accuracy', 'Total Accuracy vs Total Validation Accuracy',
            'Epochs', 'Accuracy', '3d_cnn_accuracy_plot')